In [ ]:
## 위치 인코딩
import math
import torch
from torch import nn
from matplotlib import pyplot as plt


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )

        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[: x.size(0)]
        return self.dropout(x)


encoding = PositionalEncoding(d_model=128, max_len=50)

plt.pcolormesh(encoding.pe.numpy().squeeze(), cmap="RdBu")
plt.xlabel("Embedding Dimension")
plt.xlim((0, 128))
plt.ylabel("Position")
plt.colorbar()
plt.show()

### 트랜스포머 모델 실습

torch 와 torchtext버전 호환성 문제 발생
- torchtext==0.16.0 로 맞추니까 코드 실행됨
-tochtext cache데이터도 삭제

In [12]:
!python -m spacy download de
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'de' are deprecated. Please use the
full pipeline package name 'de_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 61.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 96.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependenc

In [1]:
!pip install torchdata torchtext spacy portalocker

In [6]:
import torch
print(torch.__version__)


2.5.1+cu124


In [1]:
## 이코드 필연적 토치와 토치텍스트 버전 호환
!pip uninstall torchtext -y
!pip install torchtext==0.16.0


Found existing installation: torchtext 0.16.0
Uninstalling torchtext-0.16.0:
  Successfully uninstalled torchtext-0.16.0
  Using cached torchtext-0.16.0-cp310-cp310-manylinux1_x86_64.whl.metadata (7.5 kB)
Using cached torchtext-0.16.0-cp310-cp310-manylinux1_x86_64.whl (2.0 MB)


In [ ]:
#torch 캐시 삭제
!rm -rf ~/.cache/torchtext


In [3]:
from torchtext.datasets import Multi30k
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# 토큰 생성 함수 정의: 텍스트 iterator와 언어(독일어/영어)를 받아, 해당 언어에 맞게 토큰화된 단어들을 yield함
def generate_tokens(text_iter, language):
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}  # 언어 인덱스 설정

    for text in text_iter:
        yield token_transform[language](text[language_index[language]])

# 언어 설정: 독일어를 소스 언어, 영어를 타겟 언어로 지정
SRC_LANGUAGE = "de"
TGT_LANGUAGE = "en"

# 특별 토큰의 인덱스 설정
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ["<unk>", "<pad>", "<bos>", "<eos>"]

# 토큰화 도구 설정: 각 언어에 맞는 spaCy 모델 사용
token_transform = {
    SRC_LANGUAGE: get_tokenizer("spacy", language="de_core_news_sm"),
    TGT_LANGUAGE: get_tokenizer("spacy", language="en_core_web_sm"),
}
print("Token Transform:")
print(token_transform)  # 언어별 토큰화 함수 출력

# 각 언어별 사전을 저장할 딕셔너리 생성
vocab_transform = {}

# 각 언어에 대해 토큰을 생성하고 사전을 구축
for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    train_iter = Multi30k(split="train", language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))  # 학습용 데이터셋 로드
    vocab_transform[language] = build_vocab_from_iterator(
        generate_tokens(train_iter, language),  # 해당 언어로 토큰화된 데이터
        min_freq=1,                            # 최소 빈도 조건 설정
        specials=special_symbols,              # 특별 토큰 추가
        special_first=True,                    # 특별 토큰을 앞에 위치
    )

# UNK 토큰의 기본 인덱스를 설정하여 사전에 없는 토큰을 처리
for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[language].set_default_index(UNK_IDX)

print("Vocab Transform:")
print(vocab_transform)  # 언어별 사전 출력


Token Transform:
{'de': functools.partial(<function _spacy_tokenize at 0x7b18091c16c0>, spacy=<spacy.lang.de.German object at 0x7b17ff05ff10>), 'en': functools.partial(<function _spacy_tokenize at 0x7b18091c16c0>, spacy=<spacy.lang.en.English object at 0x7b17ff0faf80>)}
Vocab Transform:
{'de': Vocab(), 'en': Vocab()}


In [4]:
import math
import torch
from torch import nn

# PositionalEncoding: Transformer에 위치 정보를 추가하기 위한 클래스
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)  # 드롭아웃 설정

        # 위치 정보를 계산
        position = torch.arange(max_len).unsqueeze(1)  # [max_len, 1] 형태
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )  # 각 위치의 주기적 변동을 계산

        pe = torch.zeros(max_len, 1, d_model)  # 위치 인코딩 텐서 초기화
        pe[:, 0, 0::2] = torch.sin(position * div_term)  # 짝수 인덱스에 대해 sin 적용
        pe[:, 0, 1::2] = torch.cos(position * div_term)  # 홀수 인덱스에 대해 cos 적용
        self.register_buffer("pe", pe)  # 학습에서 제외되는 버퍼로 등록

    def forward(self, x):
        # 입력에 위치 인코딩을 더하고 드롭아웃 적용
        x = x + self.pe[: x.size(0)]
        return self.dropout(x)

# TokenEmbedding: 단어 토큰을 벡터로 임베딩하는 클래스
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)  # 임베딩 레이어 생성
        self.emb_size = emb_size

    def forward(self, tokens):
        # 임베딩을 루트 크기로 정규화하여 반환
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2SeqTransformer: 전체 Transformer 모델 정의
class Seq2SeqTransformer(nn.Module):
    def __init__(
        self,
        num_encoder_layers,  # 인코더 레이어 수
        num_decoder_layers,  # 디코더 레이어 수
        emb_size,            # 임베딩 크기
        max_len,             # 최대 시퀀스 길이
        nhead,               # 멀티헤드 어텐션의 헤드 수
        src_vocab_size,      # 소스 언어의 단어 수
        tgt_vocab_size,      # 타겟 언어의 단어 수
        dim_feedforward,     # 피드포워드 레이어 크기
        dropout=0.1,
    ):
        super().__init__()
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)  # 소스 임베딩 레이어
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)  # 타겟 임베딩 레이어
        self.positional_encoding = PositionalEncoding(
            d_model=emb_size, max_len=max_len, dropout=dropout
        )  # 위치 인코딩 레이어
        self.transformer = nn.Transformer(
            d_model=emb_size,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )  # Transformer 인코더-디코더 구조
        self.generator = nn.Linear(emb_size, tgt_vocab_size)  # 출력층 생성

    # 순전파 함수: 소스와 타겟 텐서, 마스크 및 패딩 마스크를 인자로 받아 모델 출력 반환
    def forward(
        self,
        src,                # 소스 시퀀스
        trg,                # 타겟 시퀀스
        src_mask,           # 소스 마스크
        tgt_mask,           # 타겟 마스크
        src_padding_mask,   # 소스 패딩 마스크
        tgt_padding_mask,   # 타겟 패딩 마스크
        memory_key_padding_mask,  # 메모리 키 패딩 마스크
    ):
        # 소스와 타겟에 임베딩 및 위치 인코딩 적용
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        # Transformer를 통해 인코딩 및 디코딩 수행
        outs = self.transformer(
            src=src_emb,
            tgt=tgt_emb,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            memory_mask=None,
            src_key_padding_mask=src_padding_mask,
            tgt_key_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=memory_key_padding_mask
        )
        return self.generator(outs)  # 최종 출력 생성

    # 인코더에서 소스 시퀀스 인코딩
    def encode(self, src, src_mask):
        return self.transformer.encoder(
            self.positional_encoding(self.src_tok_emb(src)), src_mask
        )

    # 디코더에서 타겟 시퀀스 디코딩
    def decode(self, tgt, memory, tgt_mask):
        return self.transformer.decoder(
            self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask
        )


In [5]:
from torch import optim

# 배치 크기 및 디바이스 설정
BATCH_SIZE = 128
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # GPU가 사용 가능하면 "cuda", 아니면 "cpu" 사용

# Seq2SeqTransformer 모델 초기화
model = Seq2SeqTransformer(
    num_encoder_layers=3,           # 인코더 레이어 수
    num_decoder_layers=3,           # 디코더 레이어 수
    emb_size=512,                   # 임베딩 크기
    max_len=512,                    # 최대 시퀀스 길이
    nhead=8,                        # 멀티헤드 어텐션 헤드 수
    src_vocab_size=len(vocab_transform[SRC_LANGUAGE]),  # 소스 언어의 어휘 크기
    tgt_vocab_size=len(vocab_transform[TGT_LANGUAGE]),  # 타겟 언어의 어휘 크기
    dim_feedforward=512,            # 피드포워드 레이어의 차원 크기
).to(DEVICE)  # 모델을 지정된 디바이스로 이동

# 손실 함수 설정: CrossEntropyLoss를 사용하되, 패딩 인덱스는 무시
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)

# 옵티마이저 설정: Adam을 사용하여 모델의 파라미터를 최적화
optimizer = optim.Adam(model.parameters())

# 모델 계층 구조 출력
for main_name, main_module in model.named_children():
    print(main_name)  # 최상위 모듈 이름 출력
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name)  # 첫 번째 하위 모듈 이름 출력
        for ssub_name, ssub_module in sub_module.named_children():
            print("│  └", ssub_name)  # 두 번째 하위 모듈 이름 출력
            for sssub_name, sssub_module in ssub_module.named_children():
                print("│  │  └", sssub_name)  # 세 번째 하위 모듈 이름 출력


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


src_tok_emb
└ embedding
tgt_tok_emb
└ embedding
positional_encoding
└ dropout
transformer
└ encoder
│  └ layers
│  │  └ 0
│  │  └ 1
│  │  └ 2
│  └ norm
└ decoder
│  └ layers
│  │  └ 0
│  │  └ 1
│  │  └ 2
│  └ norm
generator


In [8]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 여러 변환을 순차적으로 적용하는 함수 생성
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:  # 주어진 변환을 차례대로 적용
            txt_input = transform(txt_input)
        return txt_input
    return func

# 텍스트를 토큰 ID로 변환하고, 시작(BOS) 및 종료(EOS) 토큰을 추가
def input_transform(token_ids):
    return torch.cat(
        (torch.tensor([BOS_IDX]), torch.tensor(token_ids), torch.tensor([EOS_IDX]))
    )

# 배치 데이터를 처리하여 패딩을 추가하는 함수
def collator(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:  # 각 샘플에 대해 변환 적용
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))  # 소스 텍스트 변환
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))  # 타겟 텍스트 변환

    # 배치에 대해 패딩 추가
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch  # 패딩 처리된 텍스트 배치 반환


# 텍스트 변환 함수 설정: 토큰화, 어휘 변환, 입력 변환
text_transform = {}
for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[language] = sequential_transforms(
        token_transform[language], vocab_transform[language], input_transform
    )

# Multi30k 데이터셋에서 "valid" 데이터 로드
data_iter = Multi30k(split="valid", language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

# DataLoader 설정: 배치 크기 및 collate_fn 지정
dataloader = DataLoader(data_iter, batch_size=BATCH_SIZE, collate_fn=collator)

# 첫 번째 배치 데이터를 가져옴
source_tensor, target_tensor = next(iter(dataloader))

# 데이터를 출력하여 확인
print("(source, target):")
print(next(iter(data_iter)))  # 실제 데이터 샘플 출력

print("source_batch:", source_tensor.shape)  # 소스 배치의 크기 출력
print(source_tensor)  # 소스 배치 텐서 출력

print("target_batch:", target_tensor.shape)  # 타겟 배치의 크기 출력
print(target_tensor)  # 타겟 배치 텐서 출력


(source, target):
('Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen', 'A group of men are loading cotton onto a truck')
source_batch: torch.Size([35, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  14,    5,    5,  ...,    5,   21,    5],
        [  38,   12,   35,  ...,   12, 1750,   69],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
target_batch: torch.Size([30, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   6,    6,    6,  ...,  250,   19,    6],
        [  39,   12,   35,  ...,   12, 3254,   61],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])


In [9]:
## 어텐션 마스크 생성
# 주어진 시퀀스 길이에 대해 디코더 마스크(상삼각 행렬)를 생성하는 함수
def generate_square_subsequent_mask(s):
    # 상삼각 행렬 생성 (0보다 작은 인덱스를 masked로 처리)
    mask = (torch.triu(torch.ones((s, s), device=DEVICE)) == 1).transpose(0, 1)

    # 상삼각 행렬에서 0은 음의 무한대로, 1은 0으로 설정
    mask = (
        mask.float()
        .masked_fill(mask == 0, float("-inf"))  # 미래 단어들을 볼 수 없게 설정
        .masked_fill(mask == 1, float(0.0))    # 현재와 이전 단어는 볼 수 있게 설정
    )
    return mask

# 소스와 타겟 시퀀스에 대한 마스크를 생성하는 함수
def create_mask(src, tgt):
    src_seq_len = src.shape[0]  # 소스 시퀀스 길이
    tgt_seq_len = tgt.shape[0]  # 타겟 시퀀스 길이

    # 타겟 마스크는 디코더에서 사용할 마스크, 소스 마스크는 입력 시퀀스를 위한 마스크
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)  # 타겟의 마스크 생성
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)  # 소스는 자기 자신을 볼 수 있도록 마스크 없음

    # 소스와 타겟의 패딩을 위한 마스크 생성
    src_padding_mask = (src == PAD_IDX).transpose(0, 1)  # 소스 시퀀스에서 PAD_IDX 위치에 True 설정
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)  # 타겟 시퀀스에서 PAD_IDX 위치에 True 설정

    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

# 타겟 입력과 타겟 출력 시퀀스 준비 (타겟의 마지막 토큰은 예측해야 하므로 잘라내고, 첫 번째 토큰은 예측하기 위해 준비)
target_input = target_tensor[:-1, :]
target_out = target_tensor[1:, :]

# 소스 시퀀스와 타겟 입력에 대한 마스크 생성
source_mask, target_mask, source_padding_mask, target_padding_mask = create_mask(
    source_tensor, target_input
)

# 마스크의 형태와 내용을 출력
print("source_mask:", source_mask.shape)
print(source_mask)
print("target_mask:", target_mask.shape)
print(target_mask)
print("source_padding_mask:", source_padding_mask.shape)
print(source_padding_mask)
print("target_padding_mask:", target_padding_mask.shape)
print(target_padding_mask)


source_mask: torch.Size([35, 35])
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])
target_mask: torch.Size([29, 29])
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
         -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
         -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
         -inf, -inf, -inf, -inf, -inf],
   

In [ ]:
# 모델을 학습 모드 또는 평가 모드로 설정하고, 주어진 split에 맞는 데이터를 로드한 후, 모델을 학습 또는 평가하는 함수
def run(model, optimizer, criterion, split):
    # 모델을 학습 또는 평가 모드로 설정
    model.train() if split == "train" else model.eval()

    # 데이터셋 로드 (train 또는 valid 데이터셋)
    data_iter = Multi30k(split=split, language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    dataloader = DataLoader(data_iter, batch_size=BATCH_SIZE, collate_fn=collator)

    losses = 0  # 손실 값을 누적할 변수
    for source_batch, target_batch in dataloader:
        # 데이터를 GPU로 전송
        source_batch = source_batch.to(DEVICE)
        target_batch = target_batch.to(DEVICE)

        # 타겟 배치에서 입력과 출력을 분리 (입력은 마지막을 제외하고, 출력은 첫 번째를 제외한 나머지)
        target_input = target_batch[:-1, :]
        target_output = target_batch[1:, :]

        # 소스와 타겟의 마스크 생성
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(
            source_batch, target_input
        )

        # 모델의 출력값(logits) 계산
        logits = model(
            src=source_batch,
            trg=target_input,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            src_padding_mask=src_padding_mask,
            tgt_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=src_padding_mask,
        )

        # 옵티마이저의 기울기 초기화
        optimizer.zero_grad()

        # 손실 계산 (logits의 shape을 맞춰서 타겟 출력과 비교)
        loss = criterion(logits.reshape(-1, logits.shape[-1]), target_output.reshape(-1))

        # 훈련 모드일 경우, 역전파 및 옵티마이저 스텝을 수행
        if split == "train":
            loss.backward()
            optimizer.step()

        # 손실 값 누적
        losses += loss.item()

    # 에폭당 평균 손실 값 반환
    return losses / len(list(dataloader))

# 5 에폭 동안 모델을 훈련하고 검증 손실 출력
for epoch in range(5):
    train_loss = run(model, optimizer, criterion, "train")  # 훈련 손실
    val_loss = run(model, optimizer, criterion, "valid")   # 검증 손실
    print(f"Epoch: {epoch+1}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch: 1, Train loss: 4.465, Val loss: 3.724
Epoch: 2, Train loss: 3.559, Val loss: 3.479
Epoch: 3, Train loss: 3.308, Val loss: 3.405


In [ ]:
# 그리디 디코딩 방식으로 번역을 수행하는 함수
def greedy_decode(model, source_tensor, source_mask, max_len, start_symbol):
    # 소스 텐서와 마스크를 GPU로 이동
    source_tensor = source_tensor.to(DEVICE)
    source_mask = source_mask.to(DEVICE)

    # 모델의 인코더를 통해 소스 텍스트에 대한 메모리 계산
    memory = model.encode(source_tensor, source_mask)

    # 시작 토큰으로 초기화된 ys 텐서 생성 (크기 1x1)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)

    # 최대 길이까지 번역 생성
    for i in range(max_len - 1):
        memory = memory.to(DEVICE)

        # 타겟 마스크를 생성
        target_mask = generate_square_subsequent_mask(ys.size(0))
        target_mask = target_mask.type(torch.bool).to(DEVICE)

        # 디코더의 출력을 계산
        out = model.decode(ys, memory, target_mask)
        out = out.transpose(0, 1)  # 차원 순서를 바꿈
        prob = model.generator(out[:, -1])  # 마지막 토큰의 예측 확률을 얻음

        # 가장 확률이 높은 단어를 선택
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()  # 단어의 인덱스를 가져옴

        # 선택된 단어를 ys에 추가
        ys = torch.cat(
            [ys, torch.ones(1, 1).type_as(source_tensor.data).fill_(next_word)], dim=0
        )

        # EOS 토큰이 나오면 종료
        if next_word == EOS_IDX:
            break

    return ys


# 주어진 문장을 번역하는 함수
def translate(model, source_sentence):
    model.eval()  # 모델을 평가 모드로 설정
    # 소스 문장을 텍스트 변환기에 입력하여 텐서로 변환
    source_tensor = text_transform[SRC_LANGUAGE](source_sentence).view(-1, 1)
    num_tokens = source_tensor.shape[0]

    # 소스 문장의 마스크를 생성 (패딩 마스크)
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)

    # 그리디 디코딩을 통해 타겟 토큰을 생성
    tgt_tokens = greedy_decode(
        model, source_tensor, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX
    ).flatten()

    # 결과 토큰을 실제 단어로 변환하고, BOS와 EOS 토큰을 제외
    output = vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))[1:-1]
    return " ".join(output)


# 예시 문장 번역
output_oov = translate(model, "Eine Gruppe von Menschen steht vor einem Iglu .")  # OOV 단어 포함 예
output = translate(model, "Eine Gruppe von Menschen steht vor einem Gebäude .")  # 일반적인 문장

# 번역 결과 출력
print(output_oov)
print(output)


### GPT실습

In [ ]:
from transformers import GPT2LMHeadModel

# GPT2 모델 로드
model = GPT2LMHeadModel.from_pretrained(pretrained_model_name_or_path="gpt2")

# 모델의 모든 계층을 출력
for main_name, main_module in model.named_children():
    print(main_name)  # 주 계층의 이름 출력
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name)  # 하위 계층의 이름 출력
        for ssub_name, ssub_module in sub_module.named_children():
            print("│  └", ssub_name)  # 하위의 하위 계층의 이름 출력
            for sssub_name, sssub_module in ssub_module.named_children():
                print("│  │  └", sssub_name)  # 하위 계층의 더 깊은 하위 계층 이름 출력


In [ ]:
from transformers import pipeline

# 'text-generation' 작업을 위한 파이프라인을 생성, 모델은 'gpt2'를 사용
generator = pipeline(task="text-generation", model="gpt2")

# 텍스트 생성 수행
outputs = generator(
    text_inputs="Machine learning is",  # 텍스트 생성 시작 문장
    max_length=20,  # 생성할 최대 텍스트 길이
    num_return_sequences=3,  # 생성할 텍스트 시퀀스의 개수
    pad_token_id=generator.tokenizer.eos_token_id  # padding에 사용할 토큰 ID 설정 (끝 토큰으로 설정)
)

# 생성된 텍스트 출력
print(outputs)


### BERT 실습

In [ ]:
import numpy as np
import pandas as pd
from Korpora import Korpora


corpus = Korpora.load("nsmc")
df = pd.DataFrame(corpus.test).sample(20000, random_state=42)

In [ ]:
train, valid, test = np.split(
    df.sample(frac=1, random_state=42), [int(0.6 * len(df)), int(0.8 * len(df))]
)

print(train.head(5).to_markdown())
print(f"Training Data Size : {len(train)}")
print(f"Validation Data Size : {len(valid)}")
print(f"Testing Data Size : {len(test)}")

In [ ]:
import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler


def make_dataset(data, tokenizer, device):
    tokenized = tokenizer(
        text=data.text.tolist(),
        padding="longest",
        truncation=True,
        return_tensors="pt"
    )
    input_ids = tokenized["input_ids"].to(device)
    attention_mask = tokenized["attention_mask"].to(device)
    labels = torch.tensor(data.label.values, dtype=torch.long).to(device)
    return TensorDataset(input_ids, attention_mask, labels)


def get_datalodader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset)
    dataloader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size)
    return dataloader


epochs = 5
batch_size = 32
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path="bert-base-multilingual-cased",
    do_lower_case=False
)

train_dataset = make_dataset(train, tokenizer, device)
train_dataloader = get_datalodader(train_dataset, RandomSampler, batch_size)

valid_dataset = make_dataset(valid, tokenizer, device)
valid_dataloader = get_datalodader(valid_dataset, SequentialSampler, batch_size)

test_dataset = make_dataset(test, tokenizer, device)
test_dataloader = get_datalodader(test_dataset, SequentialSampler, batch_size)

print(train_dataset[0])

In [ ]:
from torch import optim
from transformers import BertForSequenceClassification


model = BertForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="bert-base-multilingual-cased",
    num_labels=2
).to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-5, eps=1e-8)

In [ ]:
for main_name, main_module in model.named_children():
    print(main_name)
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name)
        for ssub_name, ssub_module in sub_module.named_children():
            print("│  └", ssub_name)
            for sssub_name, sssub_module in ssub_module.named_children():
                print("│  │  └", sssub_name)

In [ ]:
import numpy as np
from torch import nn


def calc_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def train(model, optimizer, dataloader):
    model.train()
    train_loss = 0.0

    for input_ids, attention_mask, labels in dataloader:
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss = train_loss / len(dataloader)
    return train_loss

def evaluation(model, dataloader):
    with torch.no_grad():
        model.eval()
        criterion = nn.CrossEntropyLoss()
        val_loss, val_accuracy = 0.0, 0.0

        for input_ids, attention_mask, labels in dataloader:
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits

            loss = criterion(logits, labels)
            logits = logits.detach().cpu().numpy()
            label_ids = labels.to("cpu").numpy()
            accuracy = calc_accuracy(logits, label_ids)

            val_loss += loss
            val_accuracy += accuracy

    val_loss = val_loss/len(dataloader)
    val_accuracy = val_accuracy/len(dataloader)
    return val_loss, val_accuracy


best_loss = 10000
for epoch in range(epochs):
    train_loss = train(model, optimizer, train_dataloader)
    val_loss, val_accuracy = evaluation(model, valid_dataloader)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f} Val Accuracy {val_accuracy:.4f}")

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), "../models/BertForSequenceClassification.pt")
        print("Saved the model weights")

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="bert-base-multilingual-cased",
    num_labels=2
).to(device)
model.load_state_dict(torch.load("../models/BertForSequenceClassification.pt"))

test_loss, test_accuracy = evaluation(model, test_dataloader)
print(f"Test Loss : {test_loss:.4f}")
print(f"Test Accuracy : {test_accuracy:.4f}")

### BART 실습

In [ ]:
!pip install evaluate rouge_score absl-py


In [ ]:
import numpy as np
from datasets import load_dataset


news = load_dataset("argilla/news-summary", split="test")
df = news.to_pandas().sample(5000, random_state=42)[["text", "prediction"]]
df["prediction"] = df["prediction"].map(lambda x: x[0]["text"])
train, valid, test = np.split(
    df.sample(frac=1, random_state=42), [int(0.6 * len(df)), int(0.8 * len(df))]
)

print(f"Source News : {train.text.iloc[0][:200]}")
print(f"Summarization : {train.prediction.iloc[0][:50]}")
print(f"Training Data Size : {len(train)}")
print(f"Validation Data Size : {len(valid)}")
print(f"Testing Data Size : {len(test)}")

In [ ]:
import torch
from transformers import BartTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence


def make_dataset(data, tokenizer, device):
    tokenized = tokenizer(
        text=data.text.tolist(),
        padding="longest",
        truncation=True,
        return_tensors="pt",
        max_length=1024
    )
    labels = []
    input_ids = tokenized["input_ids"].to(device)
    attention_mask = tokenized["attention_mask"].to(device)
    for target in data.prediction:
        labels.append(tokenizer.encode(target, return_tensors="pt").squeeze())
    labels = pad_sequence(labels, batch_first=True, padding_value=-100).to(device)
    return TensorDataset(input_ids, attention_mask, labels)



def get_datalodader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset)
    dataloader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size)
    return dataloader


epochs = 5
batch_size = 8
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = BartTokenizer.from_pretrained(
    pretrained_model_name_or_path="facebook/bart-base"
)

train_dataset = make_dataset(train, tokenizer, device)
train_dataloader = get_datalodader(train_dataset, RandomSampler, batch_size)

valid_dataset = make_dataset(valid, tokenizer, device)
valid_dataloader = get_datalodader(valid_dataset, SequentialSampler, batch_size)

test_dataset = make_dataset(test, tokenizer, device)
test_dataloader = get_datalodader(test_dataset, SequentialSampler, batch_size)

print(train_dataset[0])

In [ ]:
from torch import optim
from transformers import BartForConditionalGeneration


model = BartForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path="facebook/bart-base"
).to(device)
optimizer = optim.AdamW(model.parameters(), lr=5e-5, eps=1e-8)

In [ ]:
for main_name, main_module in model.named_children():
    print(main_name)
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name)
        for ssub_name, ssub_module in sub_module.named_children():
            print("│  └", ssub_name)
            for sssub_name, sssub_module in ssub_module.named_children():
                print("│  │  └", sssub_name)

In [ ]:
import numpy as np
import evaluate


def calc_rouge(preds, labels):
    preds = preds.argmax(axis=-1)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    rouge2 = rouge_score.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )
    return rouge2["rouge2"]

def train(model, optimizer, dataloader):
    model.train()
    train_loss = 0.0

    for input_ids, attention_mask, labels in dataloader:
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss = train_loss / len(dataloader)
    return train_loss

def evaluation(model, dataloader):
    with torch.no_grad():
        model.eval()
        val_loss, val_rouge = 0.0, 0.0

        for input_ids, attention_mask, labels in dataloader:
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            logits = outputs.logits
            loss = outputs.loss

            logits = logits.detach().cpu().numpy()
            label_ids = labels.to("cpu").numpy()
            rouge = calc_rouge(logits, label_ids)

            val_loss += loss
            val_rouge += rouge

    val_loss = val_loss / len(dataloader)
    val_rouge = val_rouge / len(dataloader)
    return val_loss, val_rouge

rouge_score = evaluate.load("rouge", tokenizer=tokenizer)
best_loss = 10000
for epoch in range(epochs):
    train_loss = train(model, optimizer, train_dataloader)
    val_loss, val_accuracy = evaluation(model, valid_dataloader)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f} Val Rouge {val_accuracy:.4f}")

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), "../models/BartForConditionalGeneration.pt")
        print("Saved the model weights")

In [ ]:
model = BartForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path="facebook/bart-base"
).to(device)
model.load_state_dict(torch.load("../models/BartForConditionalGeneration.pt"))

test_loss, test_rouge_score = evaluation(model, test_dataloader)
print(f"Test Loss : {test_loss:.4f}")
print(f"Test ROUGE-2 Score : {test_rouge_score:.4f}")

In [ ]:
from transformers import pipeline


summarizer = pipeline(
    task="summarization",
    model=model,
    tokenizer=tokenizer,
    max_length=54,
    device="cpu"
)

for index in range(5):
    news_text = test.text.iloc[index]
    summarization = test.prediction.iloc[index]
    predicted_summarization = summarizer(news_text)[0]["summary_text"]
    print(f"정답 요약문 : {summarization}")
    print(f"모델 요약문 : {predicted_summarization}\n")

### KoELECTRA 실습

In [ ]:
import numpy as np
import pandas as pd
from Korpora import Korpora


corpus = Korpora.load("nsmc")
df = pd.DataFrame(corpus.test).sample(20000, random_state=42)
train, valid, test = np.split(
    df.sample(frac=1, random_state=42), [int(0.6 * len(df)), int(0.8 * len(df))]
)

print(train.head(5).to_markdown())
print(f"Training Data Size : {len(train)}")
print(f"Validation Data Size : {len(valid)}")
print(f"Testing Data Size : {len(test)}")

In [ ]:
import torch
from transformers import ElectraTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler


def make_dataset(data, tokenizer, device):
    tokenized = tokenizer(
        text=data.text.tolist(),
        padding="longest",
        truncation=True,
        return_tensors="pt"
    )
    input_ids = tokenized["input_ids"].to(device)
    attention_mask = tokenized["attention_mask"].to(device)
    labels = torch.tensor(data.label.values, dtype=torch.long).to(device)
    return TensorDataset(input_ids, attention_mask, labels)


def get_datalodader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset)
    dataloader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size)
    return dataloader


epochs = 5
batch_size = 32
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = ElectraTokenizer.from_pretrained(
    pretrained_model_name_or_path="monologg/koelectra-base-v3-discriminator",
    do_lower_case=False,
)

train_dataset = make_dataset(train, tokenizer, device)
train_dataloader = get_datalodader(train_dataset, RandomSampler, batch_size)

valid_dataset = make_dataset(valid, tokenizer, device)
valid_dataloader = get_datalodader(valid_dataset, SequentialSampler, batch_size)

test_dataset = make_dataset(test, tokenizer, device)
test_dataloader = get_datalodader(test_dataset, SequentialSampler, batch_size)

print(train_dataset[0])

In [ ]:
from torch import optim
from transformers import ElectraForSequenceClassification


model = ElectraForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="monologg/koelectra-base-v3-discriminator",
    num_labels=2
).to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-5, eps=1e-8)

In [ ]:
for main_name, main_module in model.named_children():
    print(main_name)
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name)
        for ssub_name, ssub_module in sub_module.named_children():
            print("│  └", ssub_name)
            for sssub_name, sssub_module in ssub_module.named_children():
                print("│  │  └", sssub_name)

In [ ]:
import numpy as np
from torch import nn


def calc_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def train(model, optimizer, dataloader):
    model.train()
    train_loss = 0.0

    for input_ids, attention_mask, labels in dataloader:
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss = train_loss / len(dataloader)
    return train_loss

def evaluation(model, dataloader):
    with torch.no_grad():
        model.eval()
        criterion = nn.CrossEntropyLoss()
        val_loss, val_accuracy = 0.0, 0.0

        for input_ids, attention_mask, labels in dataloader:
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits

            loss = criterion(logits, labels)
            logits = logits.detach().cpu().numpy()
            label_ids = labels.to("cpu").numpy()
            accuracy = calc_accuracy(logits, label_ids)

            val_loss += loss
            val_accuracy += accuracy

    val_loss = val_loss/len(dataloader)
    val_accuracy = val_accuracy/len(dataloader)
    return val_loss, val_accuracy


best_loss = 10000
for epoch in range(epochs):
    train_loss = train(model, optimizer, train_dataloader)
    val_loss, val_accuracy = evaluation(model, valid_dataloader)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f} Val Accuracy {val_accuracy:.4f}")

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), "../models/ElectraForSequenceClassification.pt")
        print("Saved the model weights")

In [ ]:
model = ElectraForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="monologg/koelectra-base-v3-discriminator",
    num_labels=2
).to(device)
model.load_state_dict(torch.load("../models/ElectraForSequenceClassification.pt"))

test_loss, test_accuracy = evaluation(model, test_dataloader)
print(f"Test Loss : {test_loss:.4f}")
print(f"Test Accuracy : {test_accuracy:.4f}")

# T5 실습

In [ ]:
import numpy as np
from datasets import load_dataset


news = load_dataset("argilla/news-summary", split="test")
df = news.to_pandas().sample(5000, random_state=42)[["text", "prediction"]]
df["text"] = "summarize: " + df["text"]
df["prediction"] = df["prediction"].map(lambda x: x[0]["text"])
train, valid, test = np.split(
    df.sample(frac=1, random_state=42), [int(0.6 * len(df)), int(0.8 * len(df))]
)

print(f"Source News : {train.text.iloc[0][:200]}")
print(f"Summarization : {train.prediction.iloc[0][:50]}")
print(f"Training Data Size : {len(train)}")
print(f"Validation Data Size : {len(valid)}")
print(f"Testing Data Size : {len(test)}")

In [ ]:
import torch
from transformers import T5Tokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence


def make_dataset(data, tokenizer, device):
    source = tokenizer(
        text=data.text.tolist(),
        padding="max_length",
        max_length=128,
        pad_to_max_length=True,
        truncation=True,
        return_tensors="pt"
    )

    target = tokenizer(
        text=data.prediction.tolist(),
        padding="max_length",
        max_length=128,
        pad_to_max_length=True,
        truncation=True,
        return_tensors="pt"
    )

    source_ids = source["input_ids"].squeeze().to(device)
    source_mask = source["attention_mask"].squeeze().to(device)
    target_ids = target["input_ids"].squeeze().to(device)
    target_mask = target["attention_mask"].squeeze().to(device)
    return TensorDataset(source_ids, source_mask, target_ids, target_mask)

def get_datalodader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset)
    dataloader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size)
    return dataloader


epochs = 5
batch_size = 8
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = T5Tokenizer.from_pretrained(
    pretrained_model_name_or_path="t5-small"
)

train_dataset = make_dataset(train, tokenizer, device)
train_dataloader = get_datalodader(train_dataset, RandomSampler, batch_size)

valid_dataset = make_dataset(valid, tokenizer, device)
valid_dataloader = get_datalodader(valid_dataset, SequentialSampler, batch_size)

test_dataset = make_dataset(test, tokenizer, device)
test_dataloader = get_datalodader(test_dataset, SequentialSampler, batch_size)

print(next(iter(train_dataloader)))
print(tokenizer.convert_ids_to_tokens(21603))
print(tokenizer.convert_ids_to_tokens(10))

In [ ]:
from torch import optim
from transformers import T5ForConditionalGeneration


model = T5ForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path="t5-small",
).to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-5, eps=1e-8)

In [ ]:
import numpy as np
from torch import nn


def calc_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def train(model, optimizer, dataloader):
    model.train()
    train_loss = 0.0

    for source_ids, source_mask, target_ids, target_mask in dataloader:
        decoder_input_ids = target_ids[:, :-1].contiguous()
        labels = target_ids[:, 1:].clone().detach()
        labels[target_ids[:, 1:] == tokenizer.pad_token_id] = -100

        outputs = model(
            input_ids=source_ids,
            attention_mask=source_mask,
            decoder_input_ids=decoder_input_ids,
            labels=labels,
        )

        loss = outputs.loss
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss = train_loss / len(dataloader)
    return train_loss


def evaluation(model, dataloader):
    with torch.no_grad():
        model.eval()
        val_loss = 0.0

        for source_ids, source_mask, target_ids, target_mask in dataloader:
            decoder_input_ids = target_ids[:, :-1].contiguous()
            labels = target_ids[:, 1:].clone().detach()
            labels[target_ids[:, 1:] == tokenizer.pad_token_id] = -100

            outputs = model(
                input_ids=source_ids,
                attention_mask=source_mask,
                decoder_input_ids=decoder_input_ids,
                labels=labels,
            )

            loss = outputs.loss
            val_loss += loss

    val_loss = val_loss / len(dataloader)
    return val_loss


best_loss = 10000
for epoch in range(epochs):
    train_loss = train(model, optimizer, train_dataloader)
    val_loss = evaluation(model, valid_dataloader)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f}")

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), "../models/T5ForConditionalGeneration.pt")
        print("Saved the model weights")

In [ ]:
model.eval()
with torch.no_grad():
    for source_ids, source_mask, target_ids, target_mask in test_dataloader:
        generated_ids = model.generate(
            input_ids=source_ids,
            attention_mask=source_mask,
            max_length=128,
            num_beams=3,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
        )

        for generated, target in zip(generated_ids, target_ids):
            pred = tokenizer.decode(
                generated, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            actual = tokenizer.decode(
                target, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            print("Generated Headline Text:", pred)
            print("Actual Headline Text   :", actual)
        break